# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
# This function opens a csv file, and transform it into a usable format 
def preprocess(filename, header):
    dataset = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile, header)
        for row in reader:
            dataset.append(row)
    return dataset

In [2]:
# This function builds a supervised NB model
def train(dataset, header, classlabels, k):
    model = []
    classes = {}
    count = 0
    #First it counts the number of times different classes appear
    for index in range(len(dataset)):
        count += 1;
        if dataset[index]['class'] in classes:
            classes[dataset[index]['class']] += 1
        else:
            classes[dataset[index]['class']] = 1
    classes['count'] = count
    model.append(classes)
    #Then it applies a smoothing technique to the rest of the data
    #Laplace smoothing if k = 1
    #Add-K if k < 1
    smoothed_attribute = smoothing(header, dataset,k) 
    i = 0
    #Then count the rest of the attributes
    while i < len(classlabels):
        current_class = classlabels[i]
        j = 0
        class_attributes = []
        while j < len(header)-1:
            attributes = smoothed_attribute[j].copy()
            for index in range(len(dataset)):
                if dataset[index]['class'] == current_class:
                    attributes['count'] += 1
                    attributes[dataset[index].get(header[j])] += 1
            class_attributes.append(attributes)
            j+=1
        model.append(class_attributes)
        i+=1
    return model

In [3]:
def smoothing(header, dataset, k):
    #smoothing is applied to the data in order to not an attribute with a count of 0  
    j = 0
    smoothed_attributes = []
    while j < len(header)-1:
        attribute = {}
        count = 0
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attribute:
                count += 1
                attribute[dataset[index][header[j]]] = k
        j+=1
        attribute['count'] = count
        smoothed_attributes.append(attribute)
    return smoothed_attributes

In [4]:
# This function predicts the class for an instance or a set of instances, based on a trained model 
def predict(model, dataset, classlabels, header):
    predicted_classes = []
    highest_probability = 0
    probability = 0
    for index in range(len(dataset)):
        row = dataset[index]
        highest_probability = 0
        probability = 0
        predicted_label = ''
        # Predict the class label
        for label in classlabels:
            # find the highest probability of a class label for an instance 
            try:
                probability = model[0][label]/model[0]['count']
                for attribute in range(len(row)-1):
                    probability *= model[classlabels.index(label)+1][attribute][row.get(header[attribute])]/model[classlabels.index(label)+1][attribute]['count']
                if probability > highest_probability:
                    highest_probability = probability
                    predicted_label = label
            # if the attribute is not found than the probability = 0
            except KeyError:
                continue
        predicted_classes.append(predicted_label)
    return predicted_classes

In [5]:
# This function evaluates a set of predictions, in a supervised context 
def evaluate(predicted_classes, dataset):
    correct = 0
    wrong = 0
    count = 0
    # Only checks how accarate the model was
    for index in range(len(dataset)):
        row = dataset[index]
        if predicted_classes[index] == row['class']:
            correct += 1
        else:
            wrong += 1
        count += 1
    print("Accuracy is", correct/count*100,"% with", correct,"correct predictions and", wrong, "wrong predictions" )

In [6]:
def class_mean_info(model, classlabels):
    classes = model[0]
    entropy_classes = 0
    i = 0
    # finds the mean info for the class labels
    while i < len(classes)-1:
        entropy_classes += classes[classlabels[i]]/classes['count']*math.log(classes[classlabels[i]]/classes['count'],2)
        i += 1
    entropy_classes = entropy_classes*(-1)
    return entropy_classes

In [7]:
def att_mean_info(dataset, attribute):
    attributes = {}
    count = 0
    for index in range(len(dataset)):
        count += 1;
        if dataset[index][attribute] in attributes:
            attributes[dataset[index][attribute]] += 1
        else:
            attributes[dataset[index][attribute]] = 1
    attributes['count'] = count
    entropy_attributes = 0
    i = 0
    # finds the mean info for the attribute
    for value in attributes.keys():
        if value != 'count':
            entropy_attributes += attributes[value]/attributes['count']*math.log(attributes[value]/attributes['count'],2)
    entropy_attributes = entropy_attributes*(-1)
    return entropy_attributes

In [8]:
def attributes_info(dataset, header, model, classlabels):
    j = 0
    attributes_values = []
    while j < len(header)-1:
        count = 0
        attributes = []
        for index in range(len(dataset)):
            if dataset[index][header[j]] not in attributes:
                attributes.append(dataset[index][header[j]])
        j+=1  
        attributes_values.append(attributes)
    attribute_index = 0
    entropy_attributes = []
    for attribute_index in  range(len(header)-1):
        mean_info = 0
        for value in attributes_values[attribute_index]:
            i = 1
            j = 1
            total = 0
            entropy = 0
            # find the total count of an attribute
            while i <= len(classlabels):
                total += model[i][attribute_index][value]-1
                i += 1
            # find the entropy of an attribute
            while j <= len(classlabels):
                if (model[j][attribute_index][value]-1)!= 0:
                    entropy += (model[j][attribute_index][value]-1)/total*math.log((model[j][attribute_index][value]-1)/total,2)
                j += 1
            entropy_attribute = entropy*(-1)
            mean_info += (total/model[0]['count'])*entropy_attribute
        entropy_attributes.append(mean_info)
    return entropy_attributes

In [29]:
# This function calculates the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(attributes_info,class_info, header):
    info_gain = []
    i = 0
    for value in attributes_info:
        info_gain.append(class_info-value)
    while i < len(header)-1:
        print("information gain for", header[i], "is", info_gain[i])
        i += 1
        

In [30]:
# This function splits the data into trainning and testing set
def holdout(train_percentage, test_percentage, dataset):
    random.shuffle(dataset)
    trainningdata = train_percentage*len(dataset)
    testingdata = test_percentage*len(dataset) + trainningdata
    train = dataset[:int(trainningdata)]
    test = dataset[int(trainningdata):int(testingdata)]
    return [train,test]
            
            

In [36]:
import csv
import math
import random

#This changes depending on the file  
filename = 'nursery.csv'
attributes = ['parents','has_nurs','form','children','housing','finance','social','health']
header = 'parents','has_nurs','form','children','housing','finance','social','health','class'
classlabels = 'not_recom', 'recommend', 'very_recom', 'priority', 'spec_prior'


#Question 1
dataset = preprocess(filename, header)
model = train(dataset, header, classlabels, 1)
predicted_classes = predict(model,dataset,classlabels, header)
evaluate(predicted_classes, dataset)
class_info1 = class_mean_info(model, classlabels)
attributes_info1 = attributes_info(dataset, header, model, classlabels)
info_gain(attributes_info1,class_info1, header) 

#Question 2
for attribute in attributes:
    print(attribute)
    entropy_attribute = att_mean_info(dataset, attribute)
    info_gain(attributes_info1,entropy_attribute, header)

#Question 4
data = holdout(0.8,0.2,dataset)
trainning_data = data[0]
testing_data = data[1]
model2 = train(trainning_data, header, classlabels, 0.02)
predicted_classes2 = predict(model2,testing_data,classlabels, header)
evaluate(predicted_classes2, testing_data)

#Question 5
model3 = train(dataset, header, classlabels, 0)
predicted_classes3 = predict(model3,dataset,classlabels, header)
evaluate(predicted_classes3, dataset)



Accuracy is 90.30092592592592 % with 11703 correct predictions and 1257 wrong predictions
information gain for parents is 0.07293460750309921
information gain for has_nurs is 0.1964492804881155
information gain for form is 0.005572591715219621
information gain for children is 0.011886431475775616
information gain for housing is 0.01960202502287145
information gain for finance is 0.0043331270252000564
information gain for social is 0.022232616894017898
information gain for health is 0.958774960469976
parents
information gain for parents is -0.05859879195953788
information gain for has_nurs is 0.0649158810254784
information gain for form is -0.12596080774741747
information gain for children is -0.11964696798686147
information gain for housing is -0.11193137443976564
information gain for finance is -0.12720027243743703
information gain for social is -0.10930078256861919
information gain for health is 0.8272415610073389
has_nurs
information gain for parents is 0.6783668022066682
informatio

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

Examples:

"cmc" Accuracy is 50.577053632043445 % with 745 correct predictions and 728 wrong predictions

information gain for w-education is 0.07090633894894594
information gain for h-education is 0.04013859922938412
information gain for n-child is 0.10173991727554088
information gain for w-relation is 0.009820501434385065
information gain for w-work is 0.002582332379721608
information gain for h-occupation is 0.030474214560266777
information gain for standard-of-living is 0.032511460053806784
information gain for media-exposure is 0.01578645559562042

"car" Accuracy is 87.15277777777779 % with 1506 correct predictions and 222 wrong predictions

information gain for buying is 0.09644896916961399
information gain for maint is 0.07370394692148596
information gain for doors is 0.004485716626632108
information gain for persons is 0.2196629633399082
information gain for lug_boot is 0.030008141247605424
information gain for safety is 0.26218435655426386

"nursery" Accuracy is 90.30092592592592 % with 11703 correct predictions and 1257 wrong predictions

information gain for parents is 0.07293460750309921
information gain for has_nurs is 0.1964492804881155
information gain for form is 0.005572591715219621
information gain for children is 0.011886431475775616
information gain for housing is 0.01960202502287145
information gain for finance is 0.0043331270252000564
information gain for social is 0.022232616894017898
information gain for health is 0.958774960469976

"anneal" Accuracy is 92.20489977728286 % with 828 correct predictions and 70 wrong predictions

information gain for family is 0.40908953764451006
information gain for product-type is 0.0
information gain for steel is 0.3060515354289405
information gain for carbon is 0.051344088764404106
information gain for hardness is 0.29108220585994726
information gain for temper_rolling is 0.14711886228095605
information gain for condition is 0.2137228803159088
information gain for formability is 0.29223544065798446
information gain for strength is 0.1261663361036096
information gain for non-ageing is 0.14107379163812883
information gain for surface-finish is 0.032488406491841815
information gain for surface-quality is 0.43517783626288564
information gain for enamelability is 0.03870173274881061
information gain for bc is 0.0004376065202120749
information gain for bf is 0.03935557414283686
information gain for bt is 0.021775078259213876
information gain for bw-me is 0.037997478813511565
information gain for bl is 0.036703081364408474
information gain for m is 0.0
information gain for chrom is 0.11722522630372056
information gain for phos is 0.02975374520863916
information gain for cbond is 0.02704235332867677
information gain for marvi is 0.0
information gain for exptl is 0.015604780443500665
information gain for ferro is 0.13718113252042574
information gain for corr is 0.0
information gain for bbvc is 0.0223970898516459
information gain for lustre is 0.01824168402125048
information gain for jurofm is 0.0
information gain for s is 0.0
information gain for p is 0.0
information gain for shape is 0.04323960556514961
information gain for oil is 0.03303757117705719
information gain for bore is 0.01937886432831948
information gain for packing is 0.003958783545891853

1: 
    The Native Bayes accuracy does seem to be correlated to the information gain of each attribute. 
    For the cmc dataset, the accuracy was 50.58% and had a total information gain of 0.303959819. While the nursery dataset had
    an accuracy of 90.3% and an information gain of 1.291785641. This accuracy difference is not due to the number of 
    attributes the dataset had because both datasets had the same number of attributes. Moreover, the car dataset had fewer 
    attributes than cmc but it had a higher accuracy of 87.15%. Its information gain was 0.686494094, which is greater than
    the one for cmc. However, datasets with more attributes are more likely to be more accurate beacause the resulting
    information gain is likely to be higher, for example, the anneal dataset has 35 attributes and has a very high accuracy of       92.2% and a information gain of 3.08758231. Thus, information gain does help to explain the classifers' behaviour. 

2:
    We assume for the Native Bayes classifier that the datasets' attributes are independent. However, there are some attributes     that break that assumption, for example, in the nursery dataset, the attribute has_nurs and health have a very high             pair-wise information gain of 1.54. Even though there are attributes that break our assumption, the overall affect on the       model appears to be very small. The accuracy of the model is at 90.3%, however, the information gain for attributes that         have a high correlation with health (has_nurs,form,children) are very low. This could be due to the fact that since they are     very positively correlated with health, the model can not gain more information from these attributes. This could affect the     accuracy of the overall model, as the lower the information gain per attribute the lower accuracy of the model will be.         However, it is unlikely that all the attributes are positive correlated with each other. 

4:
    The estimate of the model does change when using the holdout method. When the model is trained over the same data that 
    it will be evaluated with, the model is going to very accurate. Therefore, when implementing a holdout strategy, it is not
    likely to be as accurate as a model that can see all the data. Moreover, the more data that you hide from the model, the         worst it is more likely to be. For example, the anneal dataset has an accuracy of 92.2% when the model has access to all the     data but this drops down to 91.1% when 90% of data is used as the training set. However, this is not guaranteed to happen,       it is possible that the holdout splits the data in such a way that the testing data has less outliers, which can make the       the model accurate. When the model was trained on 80% of the data, its accuracy increased to 92.78%.
    

5: 
    Changing the smoothing regime does affect the accuracy of the model. With add-k smoothing the model does seem to preform         better, for example the anneal dataset model's accuracy increases from 92.2% to 93.76% when k = 0.5. However, this is           somewhat misleading as the dataset is trainned and evaluted over the same data. This means there is no reason to use             smoothing and only makes the model more inaccurate as the model takes into consideration attributes that will not appear         during the testing. Thus, with no smoothing the model becomes even more accurate, eg for anneal the model's accuracy             increases to 98.99% with k = 0. This changes when the holdout strategy is used in conjunction with add-k smoothing because       the model does need to take into consideration all possible attributes that could appear. In this case, the add-k performs       better than laplace smoothing and the lower the k, the more accurate the model becomes.